In [1]:
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report, classification 
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline

In [2]:
#read in the data and format some columns
data = pd.read_csv('warning_level_data.csv', header = 0)
data['StartDate'] = pd.to_datetime(data.StartDate)
data.WarningCode = data.WarningCode.astype(int)
#convert date to a time delta from today in days
data['TimeDelta']=(data.StartDate.apply(lambda x: (dt.datetime.today()-x).days))

In [3]:
data.head()

,LocationIdentifier,Latitude,Longitude,StartDate,Pollutant,WarningCode,WarningLevel,TimeDelta
0,11NPSWRD_WQX-PORE_319_EU-L,37.970285,-122.727744,2012-01-23,Nitrate,0,Green,1770
1,11NPSWRD_WQX-PORE_319_EU-L,37.970285,-122.727744,2012-03-13,Nitrate,0,Green,1720
2,11NPSWRD_WQX-PORE_319_EU-L,37.970285,-122.727744,2012-11-28,Nitrate,0,Green,1460
3,11NPSWRD_WQX-PORE_319_EU-L,37.970285,-122.727744,2014-02-09,Nitrate,0,Green,1022
4,11NPSWRD_WQX-PORE_319_EU-L,37.970285,-122.727744,2014-02-11,Nitrate,0,Green,1020


In [4]:
pol = data.Pollutant.unique()
pol

array(['Nitrate', 'Chromium', 'Arsenic', 'Lead', 'Copper', 'Fluoride',
       'Selenium', 'Cadmium', 'Beryllium', 'Mercury', 'Nitrite', 'Barium',
       'Antimony', 'TTHMs', 'Xylene', 'HAA5', 'PCBs', 'Simazine'], dtype=object)

In [5]:
#break into training and test

def splitData(df):
    #check how many unique sites we are measureing this pollutant at
    numLocations = df.LocationIdentifier.unique().size
    
    #pull out randomly selected entire location ids for the test set
    trainLocations = np.random.choice(df.LocationIdentifier.unique(), int(.8*numLocations), replace = False)
    testLocations = np.setdiff1d(df.LocationIdentifier.unique(), trainLocations)

    #subset the data using the train and test location identifiers
    train_data = df[df.LocationIdentifier.isin(trainLocations)]
    test_data = df[df.LocationIdentifier.isin(testLocations)]

    return train_data, test_data

In [6]:
#plots using the confusion matrix to plot precision and recall
#in order to make this useful need to increase the number of K values tried in the runKNNModel function

def PRplots(cm_dict):
    fig, axes = plt.subplots(1,3, sharey=True, figsize=(14,4))
    for k in cm_dict:
        cm = cm_dict[k]
        support = cm.sum(axis=1)
        accuracy = cm.diagonal().sum() / cm.sum().astype(float)
        recall = (cm.diagonal() / support.astype(float))
        precision = (cm.diagonal()  / cm.sum(axis=0).astype(float))
        f1 = 2*precision*recall / (precision+recall)
        for ax, r, p in zip(axes, recall, precision):
            ax.plot(k,r, marker='$R$', c=(1,0,0), markeredgecolor='none', label='recall')
            ax.plot(k,p, marker='$P$', c=(0,0,1), markeredgecolor='none', label='precision')
    [ax.set_title('Warning Code: {}'.format(i), size=16) for i,ax in enumerate(axes)];
    [ax.set_xlabel('k', size=14) for ax in axes];
    fig.tight_layout()


In [ ]:
from sklearn.grid_search import GridSearchCV

# using Ashley's function, split the data into testing and training

# find the best parameters in RF and kNN
#def best_params(model, parameters, train_data, train_labels):
def best_params(data):
    pollutant_models = {}
    RF = RandomForestClassifier()
    kNN = KNeighborsClassifier()

    for i in range(len(pol)):
        df = data[data.Pollutant == pol[i]]
        print pol[i]
        print 'data ok'
        train_data, test_data = splitData(df)
        train_labels = train_data.WarningCode
        #try:
        ### try nearest neighbors, as depending on the size of the sample
        train_points = len(train_data.index)
        #### use up to 500 nearest neighbors, if sample size permits
        size = min((train_points-5), 500)
        parameters_RF = [{"n_estimators": [k for k in range(2, size)]}]
        parameters_kNN = [{"n_neighbors": [l for l in range(2, size)]}]
        
        clf_rf = GridSearchCV(RF, parameters_RF, scoring="accuracy")
        clf_knn = GridSearchCV(kNN, parameters_kNN, scoring="accuracy")
        print 'clf OK'
        rf_model = clf_rf.fit(X=train_data[['Latitude', 'Longitude', 'TimeDelta']], y=train_data.WarningCode)
        knn_model = clf_knn.fit(X=train_data[['Latitude', 'Longitude', 'TimeDelta']], y=train_data.WarningCode)
        print 'clf fit OK'
        best_estimator_rf = clf_rf.best_estimator_
        best_score_rf = clf_rf.best_score_
        best_params_rf = clf_rf.best_params_
        best_estimator_knn = clf_knn.best_estimator_
        best_score_knn = clf_knn.best_score_
        best_params_knn = clf_knn.best_params_
            
        #print 'best_estimator OK'
        print ('Best hyperparameters: ')
        print ('RF: ', best_estimator_rf)
        print ('Best score: ', best_score_rf)
        print ('Parameters: ', str(best_params_rf))
            
        #print RF(best_estimator_rf).accuracy
        print ('kNN: ', best_estimator_knn)
        print ('Best score: ', best_score_knn)
        print ('Parameters: ', str(best_params_knn))
            
        ### save into the dictionary which model with which parameter achieved the highest accuracy,
        ### and also what is that accuracy (in case it's really low, we may want to mention it somewhere)
        if (best_score_rf > best_score_knn):
            pollutant_models[pol[i]] = {"model": rf_model, "parameter" : best_params_rf['n_estimators'], "accuracy": best_score_rf }
        else:
            pollutant_models[pol[i]] = {"model": knn_model, "parameter" : best_params_knn['n_neighbors'], "accuracy": best_score_knn }
        #pass
        #except:
            #print ('For pollutant ', pol[i], ' we don`t have enough data to try different parameters.')
        pass
    print 
    print 'RESULT'
    print pollutant_models
    return pollutant_models

### create a dictionary of models that work the best for each of the pollutants
model_dictionary = best_params(data)

Nitrate
data ok
clf OK
clf fit OK
Best hyperparameters: 
('RF: ', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=2, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))
('Best score: ', 0.65061354849554542)
('Parameters: ', "{'n_estimators': 2}")
('kNN: ', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=84, p=2,
           weights='uniform'))
('Best score: ', 0.82635737098672046)
('Parameters: ', "{'n_neighbors': 84}")
Chromium
data ok
clf OK
clf fit OK
Best hyperparameters: 
('RF: ', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, 

In [18]:
### see the list of pollutants
pol

array(['Nitrate', 'Chromium', 'Arsenic', 'Lead', 'Copper', 'Fluoride',
       'Selenium', 'Cadmium', 'Beryllium', 'Mercury', 'Nitrite', 'Barium',
       'Antimony', 'TTHMs', 'Xylene', 'HAA5', 'PCBs', 'Simazine'], dtype=object)

In [11]:
### see entries where the pollution degree is 'Red',
### and see if the model identifies them as Red too

data[data.WarningCode == 2]

,LocationIdentifier,Latitude,Longitude,StartDate,Pollutant,WarningCode,WarningLevel,TimeDelta
60,11NPSWRD_WQX-PORE_ASBS_STORM2,37.897060,-122.708720,2014-02-03,Nitrate,2,Red,1028
1445,21NEV1_WQX-NV09-301-EF-001,38.383530,-119.185799,2012-08-07,Arsenic,2,Red,1573
1830,CALWR_WQX-01N06E13B003M,37.937400,-121.264200,2012-09-12,Arsenic,2,Red,1537
1840,CALWR_WQX-01N07E26H003M,37.906000,-121.166100,2010-10-07,Nitrate,2,Red,2243
1850,CALWR_WQX-01N07E26H003M,37.906000,-121.166100,2012-09-12,Nitrate,2,Red,1537
1852,CALWR_WQX-01N08E10C001M,37.956900,-121.086600,2010-10-07,Nitrate,2,Red,2243
1862,CALWR_WQX-01N09E26A001M,37.914400,-120.948500,2012-09-12,Nitrate,2,Red,1537
1884,CALWR_WQX-01S07E21G001M,37.834400,-121.206300,2011-09-28,Nitrate,2,Red,1887
1886,CALWR_WQX-01S08E13M001M,37.848900,-121.054400,2010-10-14,Nitrate,2,Red,2236
1896,CALWR_WQX-01S08E13M001M,37.848900,-121.054400,2012-09-21,Nitrate,2,Red,1528


In [19]:
model_dictionary

{'Antimony': {'accuracy': 0.99655172413793103,
  'model': GridSearchCV(cv=None, error_score='raise',
         estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
             metric_params=None, n_jobs=1, n_neighbors=5, p=2,
             weights='uniform'),
         fit_params={}, iid=True, n_jobs=1,
         param_grid=[{'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109]}],
         pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0),
  'parameter': 2},
 'Arsenic': {'accuracy': 0.8774444146798821,
  'model': GridSearchCV(cv=None, error_score='rais

In [21]:
### for any geographic location at any time, predict the pollution level

def estimated_pollution(pollutant, latitude, longitude, timedelta):
    ### for each pollutant, recall what model and parameter do we want to use
    mod = model_dictionary[pollutant]['model']
    #par = model_dictionary[pollutant]['parameter']
    acc = model_dictionary[pollutant]['accuracy']
    #if mod == "kNN":
    #model = KNeighborsClassifier(n_neighbors = par, weights='distance')
    #model = knn.fit(X=train_data[['Latitude', 'Longitude', 'TimeDelta']], y=train_data.WarningCode)
    
    ### estimate the Warning Code
    level_pred = mod.predict([[latitude, longitude, timedelta]])
    
    if level_pred == 0:
        warning_level = 'Green'
    if level_pred == 1:
        warning_level = 'Amber'
    if level_pred == 2:
        warning_level = 'Red'
    
    print ('With ', "%0.2f" % (acc * 100 ), " probability, the warning level in your zone is: ", warning_level)
    
    return (warning_level, acc)

#test_point =  pd.DataFrame([[37.970285, 122.727744, 1], 
                      #columns=('Latitude', 'Longitude', 'TimeDelta)  

    
    ### test cases which are red
    #38.494917	-121.535056	2010-09-23	Arsenic	2	Red	2254

    
    #38.024100	-121.323800	2010-10-06	Nitrate	2	Red	2241
estimated_pollution('Arsenic', 38.494917,-121.535056, 2254)
    

('With ', '87.74', ' probability, the warning level in your zone is: ', 'Amber')


('Amber', 0.8774444146798821)

In [ ]:
# Note: currently, these models are not performing very well.